In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date
import time
import re

In [2]:
def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [3]:
url = "https://www.imdb.com/chart/top"
soup = getAndParseURL(url)
movie_urls = []
for i in soup.tbody.findAll('tr'):
    movie_urls.append('https://www.imdb.com'+i.findAll('td')[1].a['href'])

In [4]:
row = []
for i in movie_urls[:15]:
    
    html = getAndParseURL(i)

    try:
        title = html.find('h1', {"data-testid":"hero-title-block__title"}).text
    except:
        title = None
        print("Couldn't scrape the title data")

    try:
        wwgross = int(''.join(html.find('li', {"data-testid":"title-boxoffice-cumulativeworldwidegross"}).text.split('$')[1].split(',')))
    except:
        wwgross = None
        print("Couldn't scrape the gross data")

    try:
        budget = int(re.findall("\d+", ''.join(html.find('li', {"data-testid":"title-boxoffice-budget"}).li.span.text.split(' ')[0].split(',')))[0])
    except:
        budget = None
        print("Couldn't scrape the budget data")

    try:
        releaseDate = html.find('li', {"data-testid":"title-details-releasedate"}).li.a.text
    except:
        releaseDate = None
        print("Couldn't scrape the date data")

    try:
        certificate = html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[1].span.text
    except:
        certificate = None
        print("Couldn't scrape the certificate data")

    try:
        runtime = html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[2].text.split(' ')
        runtime = int(runtime[0].split('h')[0]) * 60 + int(runtime[1].split('m')[0])
    except:
        runtime = None
        print("Couldn't scrape the runtime data")

    try:
        score = float(html.find('div', {"data-testid":"hero-rating-bar__aggregate-rating__score"}).text.split('/')[0])
    except:
        score = None
        print("Couldn't scrape the score data")

    try:
        scoreN = html.find('div', {"data-testid":"hero-rating-bar__aggregate-rating"}).a.div.div.findAll('div')[1].findAll('div')[2].text
        if 'M' in scoreN:
            scoreNumber = int(float(scoreN.split('M')[0])*1_000_000)
        elif 'K' in scoreN:
            scoreNumber = int(float(scoreN.split('K')[0])*1_000)
        else:
            scoreNumber = int(scoreN)
    except:
        scoreNumber = None
        print("Couldn't scrape the score number data")

    try:
        iR = html.find('ul', {"data-testid":"reviewContent-all-reviews"}).findAll('li')[0].span.span.text
        if 'K' in iR:
            imdbReviews = int(float(iR.split('K')[0]))
        else:
            imdbReviews = int(iR)
    except:
        imdbReviews = None
        print("Couldn't scrape the imdb review data")

    try:
        criticReviews = int(html.find('ul', {"data-testid":"reviewContent-all-reviews"}).findAll('li')[1].span.span.text)
    except:
        criticReviews= None
        print("Couldn't scrape the critic review data")

    try:
        metaScore = int(html.find('ul', {"data-testid":"reviewContent-all-reviews"}).findAll('li')[2].span.span.text)
    except:
        metaScore= None
        print("Couldn't scrape the meta score data")

    try:
        directors = [] # director = html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[0].text.split('Director')[1]
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[0].findAll('a'):
            directors.append(i.text)
    except:
        directors = None
        print("Couldn't scrape the directors data")

    try:
        writers = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[1].findAll('a'):
            writers.append(i.text)
    except:
        writers = None
        print("Couldn't scrape the writers data")

    try:
        stars = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[2].find('ul', {"role":"presentation"}).findAll('a'):
            stars.append(i.text)
    except:
        stars = None
        print("Couldn't scrape the stars data")

    try:
        genres = []
        for i in html.find('li', {"data-testid":"storyline-genres"}).findAll('li'):
            genres.append(i.text)
    except:
        genres = None
        print("Couldn't scrape the genres data")

    try:
        country = html.find('li', {"data-testid":"title-details-origin"}).a.text
    except:
        country = None
        print("Couldn't scrape the country data")
    
    try:
        locations = []
        for i in html.find('li', {"data-testid":"title-details-filminglocations"}).findAll('a'):
            locations.append(i.text)
        locations = locations[1:-1]
    except:
        locations = None
        print("Couldn't scrape the locations data")

    try:
        companies = []
        for i in html.find('li', {"data-testid":"title-details-companies"}).findAll('a'):
            companies.append(i.text)
        companies = companies[1:-1]
    except:
        companies = None
        print("Couldn't scrape the companies data")

    row.append([title, wwgross, budget, releaseDate, certificate, runtime, score, scoreNumber, imdbReviews,
                   criticReviews, metaScore, directors, writers, stars, genres, country, locations, companies])
    
    print(title, "...Done✔")
    print("Sleeping 5 seconds...")
    time.sleep(5)

column = ["Title", "WorldWideGross", "Budget", "ReleaseDate", "Certificate", "Runtime", "Score",
          "ScoreReviews", "ImdbReviews", "CriticReviews", "MetaScore", "Directors", "Writers",
          "Stars", "Genres", "Country", "Locations", "Companies"]
dfMovies = pd.DataFrame.from_records(row, columns=column)

The Shawshank Redemption ...Done✔
Sleeping 5 seconds...
The Godfather ...Done✔
Sleeping 5 seconds...
The Godfather: Part II ...Done✔
Sleeping 5 seconds...
The Dark Knight ...Done✔
Sleeping 5 seconds...
12 Angry Men ...Done✔
Sleeping 5 seconds...
Schindler's List ...Done✔
Sleeping 5 seconds...
The Lord of the Rings: The Return of the King ...Done✔
Sleeping 5 seconds...
Pulp Fiction ...Done✔
Sleeping 5 seconds...
Il buono, il brutto, il cattivo ...Done✔
Sleeping 5 seconds...
The Lord of the Rings: The Fellowship of the Ring ...Done✔
Sleeping 5 seconds...
Fight Club ...Done✔
Sleeping 5 seconds...
Forrest Gump ...Done✔
Sleeping 5 seconds...
Inception ...Done✔
Sleeping 5 seconds...
The Lord of the Rings: The Two Towers ...Done✔
Sleeping 5 seconds...
Star Wars: Episode V - The Empire Strikes Back ...Done✔
Sleeping 5 seconds...


In [5]:
dfMovies.to_csv("movies.csv")
dfMovies

,Title,WorldWideGross,Budget,ReleaseDate,Certificate,Runtime,Score,ScoreReviews,ImdbReviews,CriticReviews,MetaScore,Directors,Writers,Stars,Genres,Country,Locations,Companies
0,The Shawshank Redemption,28817291,25000000,"March 10, 1995 (Turkey)",13+,142,9.3,2500000,9,193,80,[Frank Darabont],"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",[Drama],United States,"[127A Smithfield Road, Frederiksted, Virgin Is...",[Castle Rock Entertainment]
1,The Godfather,246120986,6000000,October 1973 (Turkey),18+,175,9.2,1700000,4,269,100,[Francis Ford Coppola],"[Mario Puzo, Francis Ford Coppola]","[Marlon Brando, Al Pacino, James Caan]","[Crime, Drama]",United States,"[Forza d'Agrò, Messina, Sicily, Italy]","[Paramount Pictures, Alfran Productions]"
2,The Godfather: Part II,48035783,13000000,"December 18, 1974 (United States)",18+,202,9.0,1200000,1,190,90,[Francis Ford Coppola],"[Francis Ford Coppola, Mario Puzo]","[Al Pacino, Robert De Niro, Robert Duvall]","[Crime, Drama]",United States,"[Kaiser Estate, 4000 W Lake Blvd, Homewood, La...","[Paramount Pictures, The Coppola Company, Amer..."
3,The Dark Knight,1005973645,185000000,"July 25, 2008 (Turkey)",16+,152,9.0,2500000,7,433,84,[Christopher Nolan],"[Writers, Jonathan Nolan, Christopher Nolan, D...","[Christian Bale, Heath Ledger, Aaron Eckhart]","[Action, Crime, Drama, Thriller]",United States,"[Chicago, Illinois, USA]","[Warner Bros., Legendary Entertainment, Syncopy]"
4,12 Angry Men,955,350000,"April 1, 1960 (Turkey)",15+,96,9.0,739000,1,160,96,[Sidney Lumet],[Reginald Rose],"[Henry Fonda, Lee J. Cobb, Martin Balsam]","[Crime, Drama]",United States,[New York County Courthouse - 60 Centre Street...,[Orion-Nova Productions]
5,Schindler's List,322161245,22000000,"March 4, 1994 (Turkey)",18+,195,8.9,1300000,2,170,94,[Steven Spielberg],"[Thomas Keneally, Steven Zaillian]","[Liam Neeson, Ralph Fiennes, Ben Kingsley]","[Biography, Drama, History]",United States,"[Auschwitz-Birkenau Concentration Camp, Oswiec...","[Universal Pictures, Amblin Entertainment]"
6,The Lord of the Rings: The Return of the King,1146030912,94000000,"December 19, 2003 (Turkey)",16+,201,8.9,1700000,3,361,94,[Peter Jackson],"[Writers, J.R.R. Tolkien, Fran Walsh, Philippa...","[Elijah Wood, Viggo Mortensen, Ian McKellen]","[Action, Adventure, Drama, Fantasy]",New Zealand,"[Hinuera Valley, Matamata, Waikato, New Zealand]","[New Line Cinema, WingNut Films, The Saul Zaen..."
7,Pulp Fiction,213928762,8000000,"April 14, 1995 (Turkey)",18+,154,8.9,1900000,3,302,94,[Quentin Tarantino],"[Quentin Tarantino, Roger Avary]","[John Travolta, Uma Thurman, Samuel L. Jackson]","[Crime, Drama]",United States,"[1435 Flower Street, Glendale, California, USA]","[Miramax, A Band Apart, Jersey Films]"
8,"Il buono, il brutto, il cattivo",25253751,1200000,"January 13, 1969 (Turkey)",15+,161,8.8,725000,1,148,90,[Sergio Leone],"[Writers, Luciano Vincenzoni, Sergio Leone, Ag...","[Clint Eastwood, Eli Wallach, Lee Van Cleef]","[Adventure, Western]",Italy,"[Carazo, Burgos, Castilla y León, Spain]","[Produzioni Europee Associate (PEA), Arturo Go..."
9,The Lord of the Rings: The Fellowship of the Ring,897690072,93000000,"December 21, 2001 (Turkey)",13+,178,8.8,1800000,5,201,92,[Peter Jackson],"[Writers, J.R.R. Tolkien, Fran Walsh, Philippa...","[Elijah Wood, Ian McKellen, Orlando Bloom]","[Action, Adventure, Drama, Fantasy]",New Zealand,"[Wellington, New Zealand]","[New Line Cinema, WingNut Films, The Saul Zaen..."


In [20]:
# html = getAndParseURL("https://www.imdb.com/title/tt0060196/")
# html.find('h1', {"data-testid":"hero-title-block__title"})